In [15]:
import numpy as np
import pandas
import json
import urllib
import string
from bs4 import BeautifulSoup
import os
import string
from pprint import pprint
#import seaborn as sb                                      # JR: What does the Seaborn library do? Also, needed to pip install it above
#from itertools import product                             # JR: Don't know this library either
#from matplotlib.collections import LineCollection
#import statsmodels
#import statsmodels.formula.api as smf
#from scipy.stats import chisquare
#from IPython.display import Image
thisdir = '/home/idies/workspace/Storage/raddick/afl/'
os.chdir(thisdir)
print('Done')


Done


# Get names and links for all players from afltables.com index pages

In [16]:
starting_letters = string.printable[36:62]
#print('Getting list of all players...')
all_players_list = []
for thisletter in starting_letters:
    print('Getting links to player pages for players whose last names start with {0:}...'.format(thisletter))
    thisurl = 'https://afltables.com/afl/stats/players{0:}_idx.html'.format(thisletter)
    #print(thisurl)
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    alltables = soup.findAll('table')
    allrows = alltables[0].findAll('tr')
    for these_five_players in allrows[1:]:
        for this_player in these_five_players.findAll('td'):
            playerdict = {}
            playerdict['name_full_commas'] = this_player.text
            try:
                playerdict['link'] = this_player.find('a').get('href')
                playerdict['link_title'] = this_player.find('a').get('title')
                all_players_list.append(playerdict)
            except AttributeError:
                pass
print('Parsing names and fixing URLs...')
for this_player in all_players_list:
    this_player['name_last'] = this_player['name_full_commas'].split(',')[0].strip()
    this_player['name_first'] = this_player['name_full_commas'].split(',')[1].strip()
    this_player['link'] = 'https://afltables.com/afl/stats/'+this_player['link']

print('creating data frame of all players...')
players_df = pandas.DataFrame(all_players_list)
column_order = ['name_last', 'name_first', 'link', 'link_title', 'name_full_commas']
players_df = players_df[column_order]

print('backing up...')
players_df_bk = players_df

print('Got links to all {0:,.0f} players from afltables.com!'.format(len(players_df)))

Getting links to player pages for players whose last names start with A...
Getting links to player pages for players whose last names start with B...
Getting links to player pages for players whose last names start with C...
Getting links to player pages for players whose last names start with D...
Getting links to player pages for players whose last names start with E...
Getting links to player pages for players whose last names start with F...
Getting links to player pages for players whose last names start with G...
Getting links to player pages for players whose last names start with H...
Getting links to player pages for players whose last names start with I...
Getting links to player pages for players whose last names start with J...
Getting links to player pages for players whose last names start with K...
Getting links to player pages for players whose last names start with L...
Getting links to player pages for players whose last names start with M...
Getting links to player p

# Create player IDs

## Look for duplicate player ID strings (4 letters last name + 1 first name)

In [17]:
print('retrieving from backup...')
players_df = players_df_bk

print('getting list of teams played for...')
players_df = players_df.assign(teams_list = players_df['link_title'].apply(lambda x: x.split(',')))
print('getting first year in the league...')
players_df = players_df.assign(first_year = players_df['teams_list'].apply(lambda x: int(x[0].split('(')[1].split('-')[0].replace(')', '').strip())))
print('getting last year in the league...')
players_df = players_df.assign(last_year = players_df['teams_list'].apply(lambda x: int(x[-1].split('(')[1].split('-')[-1].replace(')','').strip())))

print('starting to assign player ids...')
players_df = players_df.assign(almost_id = players_df['name_last'].apply(lambda x: x[0:4].lower())+players_df['name_first'].apply(lambda x: x[0:1].lower()))
#players_df
print('looking for duplicate player ids...')


duplicate_players_df = players_df[['almost_id', 'first_year']][players_df['almost_id'].duplicated(keep=False)].sort_values(by=['almost_id', 'first_year'])

duplicate_players_df = duplicate_players_df.assign(occur = np.nan)

duplicate_players_df = duplicate_players_df.reset_index()
duplicate_players_df = duplicate_players_df.rename(columns={'index': 'players_df_index'})

duplicate_players_df.loc[0, 'occur'] = 1
occur = 1
for newix, thisrow in duplicate_players_df[1:].iterrows():
    if (duplicate_players_df['almost_id'].loc[newix] == duplicate_players_df['almost_id'].loc[newix-1]):
        occur = occur + 1
    else:
        occur = 1
    duplicate_players_df.loc[newix, 'occur'] = occur

duplicate_players_df = duplicate_players_df.set_index('players_df_index', drop=True)

print('backing up...')
players_df_bk = players_df
duplicate_players_df_bk = duplicate_players_df

print('ok')



retrieving from backup...
getting list of teams played for...
getting first year in the league...
getting last year in the league...
starting to assign player ids...
looking for duplicate player ids...
backing up...
ok


## Assign retrosheet-style playerIDs to each player

In [18]:
print('retrieving from backup...')
players_df = players_df_bk
duplicate_players_df = duplicate_players_df_bk

players_df = players_df.assign(occur = 1)
players_df.loc[duplicate_players_df.index, 'occur'] = duplicate_players_df['occur']
players_df['occur'] = pandas.to_numeric(players_df['occur'], downcast='integer', errors='coerce')

players_df = players_df.assign(player_id = np.nan)
players_df.loc[players_df['last_year'] >= 1984, 'player_id'] = players_df[players_df['last_year'] >= 1984].apply(lambda row: row['almost_id']+'0'+'{:02d}'.format(row['occur']), axis=1)
players_df.loc[players_df['last_year'] < 1984, 'player_id'] = players_df[players_df['last_year'] < 1984].apply(lambda row: row['almost_id']+'1'+'{:02d}'.format(row['occur']), axis=1)

players_df = players_df.set_index('player_id')
players_df = players_df.drop(['almost_id', 'occur'], axis=1)
print('saving file...')
players_df.to_csv('all_player_names.csv', encoding='utf-8')
print('Done!')


retrieving from backup...
saving file...
Done!


# Save list of player IDs and names

,name_last,name_first,link,link_title,name_full_commas,teams_list,first_year,last_year
player_id,,,,,,,,
aanev101,Aanensen,Vic,https://afltables.com/afl/stats/players/V/Vic_...,Sydney (1973-1976),"Aanensen, Vic",[Sydney (1973-1976)],1973,1976
abbea101,Abbey,Angus,https://afltables.com/afl/stats/players/A/Angu...,Western Bulldogs (1949-1954),"Abbey, Angus",[Western Bulldogs (1949-1954)],1949,1954
abber101,Abbey,Ross,https://afltables.com/afl/stats/players/R/Ross...,Western Bulldogs (1971-1981),"Abbey, Ross",[Western Bulldogs (1971-1981)],1971,1981
abboc102,Abbott,Charles,https://afltables.com/afl/stats/players/C/Char...,Hawthorn (1961-1963),"Abbott, Charles",[Hawthorn (1961-1963)],1961,1963
abboc101,Abbott,Clarrie,https://afltables.com/afl/stats/players/C/Clar...,"Collingwood (1907), Melbourne (1912)","Abbott, Clarrie","[Collingwood (1907), Melbourne (1912)]",1907,1912
abboe101,Abbott,Ernie,https://afltables.com/afl/stats/players/E/Erni...,St Kilda (1908),"Abbott, Ernie",[St Kilda (1908)],1908,1908
abbof101,Abbott,Frank,https://afltables.com/afl/stats/players/F/Fran...,Fitzroy (1906-1912),"Abbott, Frank",[Fitzroy (1906-1912)],1906,1912
abbog101,Abbott,Gordon,https://afltables.com/afl/stats/players/G/Gord...,"Geelong (1936-1938), Essendon (1941-1947)","Abbott, Gordon","[Geelong (1936-1938), Essendon (1941-1947)]",1936,1947
abbol101,Abbott,Les,https://afltables.com/afl/stats/players/L/Les_...,"Collingwood (1904), Carlton (1905), Richmond (...","Abbott, Les","[Collingwood (1904), Carlton (1905), Richmon...",1904,1912


# Get game stats for each player

In [ ]:
debug = 0

print('retrieving from backup...')
players_df = players_df_bk
allplayers = []
startrow = 1000
endrow = 2000
cnt = startrow
all_players_list = []
for ix, thislink in players_df['link'][startrow:endrow].iteritems():  # ITERATE PLAYERS
    if (debug == 1):
        print('Row {0:.0f}: Player ID {1:}: {2:}'.format(cnt, ix, thislink))
    else:
        if (np.mod(cnt, 50) == 0):
            print('Row {0:.0f}: Player ID {1:}: {2:}'.format(cnt, ix, thislink))
    thispage = urllib.request.urlopen(thislink)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    alltables = soup.findAll('table')
    this_player_stats_list = []
    for thistable in alltables:    # ITERATE SEASONS/TEAMS
        badtable = False
        thishead = thistable.find('thead')
        headrows = thistable.find('thead').findAll('tr')
        thisteam = headrows[0].find('th').text.split('-')[0].strip()
        try:
            thisyear = int(headrows[0].find('th').text.split('-')[1].strip())
        except IndexError:
            badtable = True
        if (badtable == False):
            if (debug >= 1):
                print('{0:} {1:.0f}...'.format(thisteam,thisyear))
            this_season_stats_list = []
            datacols = []
            dataheaders = headrows[1].findAll('th')
            for thiscol in dataheaders:
                datacols.append(thiscol.text)
            bodyrows = thistable.findAll('tr')
            for thisbodyrow in bodyrows[2:-1]:   # ITERATE GAMES
                this_game_stats_dict = {}
                this_game_stats_dict['team'] = thisteam
                game_data_text_all = thisbodyrow.findAll('td')
                for j in range(0, len(game_data_text_all)):
                    if (game_data_text_all[j].text == '\xa0'):
                        this_game_stats_dict[datacols[j]] = 0
                    elif (datacols[j] != 'Opponent' and datacols[j] != 'R' and datacols[j] != 'Rd'):
                        try:
                            this_game_stats_dict[datacols[j]] = int(game_data_text_all[j].text)
                        except ValueError:
                            this_game_stats_dict[datacols[j]] = int(game_data_text_all[j].text[0:-1])
                    else:
                        this_game_stats_dict[datacols[j]] = game_data_text_all[j].text
                this_game_stats_bigdict = {}
                this_game_stats_bigdict[this_game_stats_dict['Rd']] = this_game_stats_dict
                del this_game_stats_bigdict[this_game_stats_dict['Rd']]['Gm']
                del this_game_stats_bigdict[this_game_stats_dict['Rd']]['Rd']
                this_season_stats_list.append(this_game_stats_bigdict)
            this_season_stats_bigdict = {}
            this_season_stats_bigdict[thisyear] = this_season_stats_list
            this_player_stats_list.append(this_season_stats_bigdict)
    this_player_stats_bigdict = {}
    this_player_stats_bigdict[ix] = this_player_stats_list
    all_players_list.append(this_player_stats_bigdict)
    cnt = cnt + 1
         #   pprint(this_season_stats_bigdict)
        #this_player_stats_list.append(this_season_stats_bigdict)
        #pprint(this_player_stats_list)
    #this_player_stats_list.append(this_season_stats_bigdict)
    #pprint(this_player_stats_list)
    #print('\n\n')
#            this_game_stats_bigdict[thisyear] = this_game_stats_dict
#        this_player_stats_list.append(this_game_stats_bigdict)
#    this_player_stats_bigdict = {}
#    this_player_stats_bigdict[ix] = this_player_stats_list
print('Done!')
#pprint(all_players_list)


In [ ]:
with open('players_2k.json', 'w') as fp:
    json.dump(all_players_list, fp)
print('done')

In [ ]:
with open('players_1k.json', 'r') as f:
    x = json.load(f)
pprint(x)

In [ ]:

            
                
                
                this_game_stats_dict = {}
                this_game_stats_dict[this_game_stats['Rd']] = this_game_stats
                del this_game_stats['Rd']
                del this_game_stats['Gm']
                #print(this_game_stats_dict)
                this_season_stats.append(this_game_stats_dict)
            this_season_stats_dict = {}
            this_season_stats_dict[thisyear] = this_season_stats
        this_player_stats.append(this_season_stats_dict)
        this_player_stats_dict = {}
        this_player_stats_dict[ix] = this_player_stats
    allplayers.append(this_player_stats_dict)
    cnt = cnt + 1
print(len(allplayers))
#pprint(this_player_stats_dict)
#pprint(allplayers)

In [ ]:
for thisone in allplayers:
    for pid, pdata in thisone.items():
        print('Player {0:}'.format(pid))
        for i in range(0, len(pdata)):
            for sid, sdata in pdata[i].items():
                print('Season {0:}'.format(sid))
                for j in range(0, len(sdata)):
                    for gid, gdata in sdata[j].items():
                        print('Game {0:}'.format(gid))
                        pprint(gdata)
                        print('\n')
            

In [ ]:
datacols = ['game_number', 'opponent', 'round', 'result', 'jersey_number', 'kicks', 'marks', 'handballs']
datacols += ['disposals', 'goals', 'behinds', 'hitouts', 'tackles', 'rebound50s', 'inside50s']
datacols += ['clearances', 'clangers', 'free_kicks_for', 'free_kicks_against', 'brownlow_votes']
datacols += ['contested_possessions', 'uncontested_possessions', 'contested_marks']
datacols += ['one_percenters', 'bounces', 'goal_assists', 'pct_game_played']

df = pandas.DataFrame(data=None, columns=datacols)
for i in range(0,1):#len(all_player_links)):
    #print(all_player_links[i])
    thisplayername = all_player_links[i].split('/')[-1][0:-5].replace('_',' ')
    thisurl = all_player_links[i]
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    print('Parsing player {0:,.0f}: {1:}...'.format(i, thisplayername))
    alltables = soup.findAll('table')
    thisyearstats = []
    for j in range(7, len(alltables)):
        thisyeardata = alltables[j]
        thisgamestats = {}
        print('Parsing data from {0:}...'.format(alltables[j].findAll('tr')[0].text[-4:]))
        allrows = thisyeardata.findAll('tr')[2:-1]
        for thisrow in allrows:
            allcells = thisrow.findAll('td')
            for k in range(0, len(allcells)-1):
                thisgamestats[datacols[k]] = allcells[k].text
            thisyearstats.append(thisgamestats)
        print(thisyearstats)

In [ ]:

movies = []
colnames = ['Release Date','Movie','Genre','Production Budget','Domestic Box Office to Date','Trailer']
nColumns = len(colnames)
starting_letters = string.printable[0:36]+string.printable[69]
#starting_letters = ['a']
for thisletter in starting_letters:
    thisurl = 'http://www.the-numbers.com/movies/letter/1'#+thisletter
    print(thisurl)
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    firstTable = soup.find('table')
    theseRows = soup.findAll('tr')
    for row in theseRows:
        thisrow = []
        cells = row.findAll('td')
        for cell in cells:
            if (cell.get_text() == u'\xa0'):
                thisrow.append(''.encode('utf-8'))
            else:
                thisrow.append(cell.get_text().replace(u'\xa0',' ').encode('utf-8'))
        if (len(thisrow) == 6):
            movies.append(thisrow)
movies_decoded = [[s.decode('utf-8') for s in t] for t in movies]
print('Done!')

In [ ]:
thenumbers = pd.DataFrame.from_dict(movies_decoded)
thenumbers.columns = ['releasedate','title','genre','budget','revenue','trailer']

thenumbers['releasedate'] = pd.to_datetime(thenumbers['releasedate'],format=u"%b\xa0%d,\xa0%Y")#,coerce=True)

thenumbers = thenumbers.replace([u'',u'nan'],[np.nan,np.nan])

thenumbers['budget'] = thenumbers['budget'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['budget'] = thenumbers['budget'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['revenue'] = thenumbers['revenue'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['revenue'] = thenumbers['revenue'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['year'] = thenumbers['releasedate'].apply(lambda x:x.year)

thenumbers['has_trailer'] = thenumbers['trailer'].apply(lambda x: True if (x == 'Play') else False)

thenumbers = thenumbers.drop('trailer',axis=1)

thenumbers.index.name = 'id'

thenumbers.columns = ['releasedate_thenumbers','title_thenumbers','genre_thenumbers','budget_thenumbers','revenue_thenumbers','year_thenumbers','has_trailer_thenumbers']
thenumbers.head(30)

thenumbers.to_csv('revenue.csv',encoding='utf8')
print(thenumbers.shape)  # as of 12/6/2017: 30,547 rows x 7 columns
                         # as of 4/12/2018: 31,711 rows x 7 columns
print('Done')